In [ ]:
import pandas as pd
from slugify import slugify

from pendulum import now

In [ ]:
dataset_name = 'obscana_etanol'
raiz = '../databases'

### Bronze

In [ ]:
ano_inicio = 1990
ano_fim = now().year - 1

# ano_inicio = now().year - 2
# ano_fim = now().year - 1

url = f'https://observatoriodacana.com.br/historico-de-producao-e-moagem.php?idMn=31&tipoHistorico=2&acao=visualizar&idTabela=2492&produto=etanol_total&safraIni={ano_inicio}%2F{ano_inicio+1}&safraFim={ano_fim}%2F{ano_fim+1}&estado=RS%2CSC%2CPR%2CSP%2CRJ%2CMG%2CES%2CMS%2CMT%2CGO%2CDF%2CBA%2CSE%2CAL%2CPE%2CPB%2CRN%2CCE%2CPI%2CMA%2CTO%2CPA%2CAP%2CRO%2CAM%2CAC%2CRR'

bronze_df = pd.read_html(f'{url}', thousands = ".", decimal = ",")[0]

# bronze_df.to_html(f'{raiz}/bronze_{dataset_name}.html', index=False)
bronze_df

### Prata

In [ ]:
estados_df = (
    pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
        .filter(['UF-id', 'UF-nome'])
        .drop_duplicates()
        .set_index("UF-nome")     
)

estados_df

In [ ]:
anos_preditos = list(range(2020 + 1, ano_fim))

dict(list(zip(anos_preditos, ["série"] * len(anos_preditos))))

In [ ]:
silver_df = (
    bronze_df
        .query('not Estado.str.contains("Região.*|Brasil", case=False)')    
        .rename(columns=lambda col: col if col == 'Estado' else col[0:4]) # pegar o ano anterior
        .assign(**{
            "2021": lambda df: df["2020"],
            "2022": lambda df: df["2020"],
            "2023": lambda df: df["2020"],
            "2024": lambda df: df["2020"],
        })
        # 
        .set_index("Estado")
        .join(estados_df)
        .assign(**{
            'fonte': 'Fertilizante orgânico vinhaça'
        })
        .set_index(["UF-id",'fonte'])
        .stack()
        .rename_axis(["uf", "fonte", "ano"], axis="rows")
        .reorder_levels([0, 2, 1])
        .sort_index()
        .rename("safra_mil_m3")    
)

# silver_df.to_frame().to_parquet(f'{raiz}/prata_{dataset_name}.parquet')

silver_df

### Ouro

Produção de Etanol ao todo por Estado (1000 m³)

Parâmetros
Teor (kg N/m³)

Cálculo: ((produção estadual * producao)*(parametro_X))*(1000/1000)

Brasil = mil t

NPK na forma do fertilizante orgânico vinhaça incorporado ao solo (t)

In [ ]:
producao = 11.5

parametro_N = 0.433
parametro_P = 0.034
parametro_K = 2.206

In [ ]:
gold_df = (
    # pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
    silver_df.to_frame()
        .assign(
            vinhaca = lambda df: df['safra_mil_m3'] * producao
        )
        .assign(**{
            'N': lambda df: (df['vinhaca'] * parametro_N), # *(1000/1000),
            'P': lambda df: (df['vinhaca'] * parametro_P), # *(1000/1000),
            'K': lambda df: (df['vinhaca'] * parametro_K), # *(1000/1000)
        })
        .reset_index()
        .melt(
            id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor'
        )
        .set_index(['uf', 'ano', 'fonte', 'elemento'])
        .sort_index()
        .squeeze()
)

gold_df.to_csv(f'{raiz}/ouro_npk_fert_organico_vinhaca.csv')
gold_df